In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 56.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=aae738722bb4cb4bf1ecae3ea341759117cfd069cd59d1593664b882020ed5d9
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [4]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import numpy as np
import os
import random

# Set random seeds for reproducibility
random.seed(0)
os.environ['PYTHONHASHSEED'] = str(0)
np.random.seed(0)
torch.manual_seed(0)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU...")
else:
    device = torch.device("cpu")
    print("Running on CPU...")

if device.type == 'cuda':
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)

# Hyperparameters
config = dict(
    epochs=300,
    classes=10,
    train_batch_size=32,
    val_batch_size=10,
    test_batch_size=10,
    steps=0,
    print_every=23,
    best_val_accuracy=0,
    curr_epoch=0,
    curr_val_acc=0,
    curr_lr=0,
    lr=1e-4,
    naming_num=0,
    dataset="10 Class Rock Data",
    architecture="EfficientNet",
    optimizer='adam'
)


def model_pipeline(hyperparameters):
    with wandb.init(project="pytorch-demo", config=hyperparameters):
        config = wandb.config
        train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomVerticalFlip(),
                                          #  transforms.RandAugment(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
        dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Kratos_ML/data/train', transform=train_transforms)

        model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
        train(model, train_loader, val_loader, criterion, optimizer, config)
        test(model, test_loader, config)

    return model

def make(config):
    # Load EfficientNet model
    model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=config['classes'])

    # Replace classifier
    classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(model._fc.in_features, 512)),
        ('relu1', nn.LeakyReLU()),
        ('fc2', nn.Linear(512, config['classes']))
    ]))
    model._fc = classifier

    criterion = nn.CrossEntropyLoss()

    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    elif config['optimizer'] == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    model.to(device)

    print("Done")
    train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomVerticalFlip(),
                                          #  transforms.RandAugment(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
    dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Kratos_ML/data/train', transform=train_transforms)
    train_loader, val_loader, test_loader = make_loader(dataset, config['train_batch_size'])
    return model, train_loader, val_loader,test_loader, criterion, optimizer

def make_loader(dataset, batch_size, split_ratio=0.8):
    # Split the dataset into train, validation, and test sets
    train_size = int(split_ratio * len(dataset))
    val_size = int((len(dataset) - train_size) / 2)
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    train_transforms = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    test_transforms = transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_dataset.transform = train_transforms
    val_dataset.transform = val_transforms
    test_dataset.transform = test_transforms

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2,
                              pin_memory=True, drop_last=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=2,
                            pin_memory=True, drop_last=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2,
                             pin_memory=True, drop_last=False)

    return train_loader, val_loader, test_loader


def test(model, test_loader, config):
    model.eval()
    accuracy = 0
    running_f1 = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model.forward(inputs)
            ps = torch.softmax(logits, dim=1)

            top_p, top_class = ps.topk(1, dim=1)
            correct_preds = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(correct_preds.type(torch.FloatTensor)).item()
            running_f1 += f1_score(labels.cpu().numpy(), top_class.cpu().numpy(), average='macro')

    test_accuracy = accuracy / len(test_loader)
    test_f1 = running_f1 / len(test_loader)

    wandb.log({"Test Accuracy": test_accuracy, "Test F1 Score": test_f1}, step=config.steps)
    print(f"Test Accuracy: {test_accuracy:.3f} | Test F1 Score: {test_f1:.3f}")

def train(model, train_loader, val_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)

    total_batches = len(train_loader) * config.epochs
    running_loss = 0

    for epoch in range(config.curr_epoch, config.epochs):
        for inputs, labels in train_loader:
            config.steps += 1
            running_loss += train_batch(inputs, labels, model, optimizer, criterion)

            if config.steps % config.print_every == 0:
                val_loss = 0
                accuracy = 0
                running_f1 = 0
                predicted = ()
                model.eval()

                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logits = model.forward(inputs)
                        ps = torch.softmax(logits, dim=1)
                        batch_loss = criterion(ps, labels)
                        val_loss += batch_loss.item()

                        top_p, top_class = ps.topk(1, dim=1)
                        correct_preds = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(correct_preds.type(torch.FloatTensor)).item()
                        running_f1 += f1_score(labels.cpu().numpy(), top_class.cpu().numpy(), average='macro')

                config.curr_epoch = epoch + 1
                curr_train_loss = running_loss / config.print_every
                curr_val_loss = val_loss / len(val_loader)
                config.curr_val_acc = accuracy / len(val_loader)
                config.curr_lr = optimizer.param_groups[0]['lr']
                curr_f1 = (running_f1 / len(val_loader))

                train_log(config.curr_epoch, curr_train_loss, curr_val_loss, config.curr_val_acc, config.curr_lr, curr_f1, config)
                running_loss = 0
                model.train()

            if config.curr_val_acc > config.best_val_accuracy:
                config.best_val_accuracy = config.curr_val_acc
                if config.best_val_accuracy > 0.58:
                    print(f"Saving new best model at {config.best_val_accuracy:.4f}... \n")
                    torch.save(model.state_dict(), f'/content/drive/MyDrive/Kratos_ML/kratos/models2/efficientnet/best_model_{config.best_val_accuracy}.pth')

            if (((config.curr_epoch % 20) == 0) and (config.steps % 47 == 0)):
                print(f"Saving checkpoint at epoch: {config.curr_epoch}... \n")
                torch.save(model.state_dict(), f'/content/drive/MyDrive/Kratos_ML/kratos/models2/efficientnet/epoch_{config.curr_epoch}_acc_{config.curr_val_acc}.pth')

def train_batch(inputs, labels, model, optimizer, criterion):
    inputs, labels = inputs.to(device), labels.to(device)

    logits = model.forward(inputs)
    ps = torch.softmax(logits, dim=1)
    loss = criterion(ps, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

def train_log(curr_epoch, curr_train_loss, curr_val_loss, curr_val_acc, curr_lr, curr_f1, config):
    wandb.log({"Epoch": curr_epoch, "Train loss": curr_train_loss, "Val loss": curr_val_loss, "Val Accuracy": curr_val_acc,
               "Current Learning Rate": curr_lr, "Val F1 Score": curr_f1}, step=config.steps)
    print(f"Epoch {curr_epoch}/{config.epochs}.. "
          f"Train loss: {curr_train_loss:.3f}.. "
          f"Val loss: {curr_val_loss:.3f}.. "
          f"Val accuracy: {curr_val_acc:.3f}.. "
          f"Learning rate: {curr_lr}.. "
          f"F1 Score: {curr_f1:.3f}")

model = model_pipeline(config)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: f20220692. Use `wandb login --relogin` to force relogin


Running on CPU...


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 184MB/s]


Loaded pretrained weights for efficientnet-b0
Done
Epoch 1/300.. Train loss: 2.294.. Val loss: 2.291.. Val accuracy: 0.267.. Learning rate: 0.0001.. F1 Score: 0.158
